In [ ]:
import pandas as pd 
import datetime
import numpy as np

# Data processing. Trajectories

This notebook contains the necessary steps to process and filter the meteorological geolocated data collected via Meotracker sensors during the thermal walks. The notebook also contains an example of application for a given dataframe (csv file).

**Resume:** 

The first step is to keep only the informative and non-empty columns. In the second step, linear interpolation is applied to the data to resample all records to a uniform temporal resolution of one measurement per second. Then, in the third step, the data from two sensors mounted on the same cart are averaged at every timestamp. Finally, in the fourth and last step a new shifted temperature is computed by subtracting, at each record, the temperature recorded by the fixed-station sensor located at the starting (and ending) point of the trajectory. This is done to mitigate the effect of natural temperature variation over the course of the day. The same procedure is applied to the humidex (HDX), which combines the temperature and the relative humidity. The resulting dataset for each trajectory (thermal walk) has 14 columns.

The fixed reference sensors at the starting/ending point of the walk are processed in the same way excluding step number 4 (thus each fixed sensor dataset has 10 columns instead of 14).

**Index:**

    1. Remove columns
    2. Linear interpolation
    3. Average two sensors of the same cart
    4. Shifted temperatures and humidex
    
   


<br>

## 1. Remove columns

Each individual dataset initially contained 28 columns. Most of these columns were empty or corresponded to functionalities not supported by the MeteoTracker sensor model we used. Therefore, we retained only the 10 relevant columns: timestamp, latitude, longitude, temperature, relative humidity, altitude, preassure, dew-point, humidex and speed.



In [ ]:
def remove_columns(df):
    '''
    Function that removes irrelevant and empty columns from a given dataframe,
    keeping only the 10 relevant ones.
    
    Parameters:
        df (pd.DataFrame): Input dataframe

    Returns:
        pd.DataFrame: Dataframe with only the relevant columns
    '''
    keep_columns = ['Time', 'Lat', 'Lon', 'Temp[°C]', 'Hum[%]', 'Alt[m]',
                    'Press[mbar]', 'DP[°C]', 'HDX[°C]', 'Speed[km/h]']
    
    
    # Keep only the columns that are both in the DataFrame and in keep_columns
    relevant_df = df[[col for col in keep_columns if col in df.columns]]
    
    return relevant_df

    

<br>

## 2. Linear interpolation
Each individual dataset trajectory is linearly interpolated to have all records (from all columns) with constant periodicity of one second. 


In [ ]:
def linear_interpolation(df):
    '''
    Function that interpolates linearly the data to have each record separated by 1 second.
    
    It fills the time gaps of more than one second using linear interpolation
    on the continuous numeric variables (temperature, humidity, etc.).
    
    Parameters:
        df (pd.DataFrame): Input dataframe with a 'Time' column (datetime format)

    Returns:
        pd.DataFrame: Interpolated dataframe with 1-second frequency
    '''
    print(f'The original data-set has {len(df)} records')

    # Ensure 'Time' is datetime
    df['Time'] = pd.to_datetime(df['Time'])
    
    # Set 'Time' as index
    df = df.set_index('Time')


    # Resample to 1-second intervals and interpolate linearly
    df = df.resample('1S').asfreq().interpolate(method='linear')

    # Reset index to bring 'Time' back as a column
    df.reset_index(inplace=True)

    print(f'After interpolating, the new data-set has {len(df)} records\n')
    
    return df



## 3. Average two sensors of the same cart
Given two datasaets corresponding to two sensors mounted on the same cart, we average the data of the overlapping time of both sensors. 

In [ ]:
def average_2_sensors(df1, df2):
    '''
    Function that computes the average of two synchronized sensor datasets from the same trolley.
    
    Steps:
    1. Aligns both dataframes to the common time range (intersection).
    2. Verifies they have the same number of records.
    3. Computes the mean value of all shared numeric columns.
    
    Parameters:
        df1 (pd.DataFrame): First sensor dataframe, must contain 'Time' column.
        df2 (pd.DataFrame): Second sensor dataframe, must contain 'Time' column.
        
    Returns:
        pd.DataFrame: Averaged dataframe over shared time interval.
    '''
    
    # Ensure Time is datetime
    df1['Time'] = pd.to_datetime(df1['Time'])
    df2['Time'] = pd.to_datetime(df2['Time'])

    # Find overlapping time range
    start_time = max(df1['Time'].min(), df2['Time'].min())
    end_time = min(df1['Time'].max(), df2['Time'].max())

    print(f'Dataset 1: {len(df1)} records, from {df1["Time"].min()} to {df1["Time"].max()}')
    print(f'Dataset 2: {len(df2)} records, from {df2["Time"].min()} to {df2["Time"].max()}')
    print(f'Overlap: from {start_time} to {end_time}')

    # Trim both dataframes to overlapping range
    df1_trimmed = df1[(df1['Time'] >= start_time) & (df1['Time'] <= end_time)].reset_index(drop=True)
    df2_trimmed = df2[(df2['Time'] >= start_time) & (df2['Time'] <= end_time)].reset_index(drop=True)

    # Verify matching record count
    if len(df1_trimmed) != len(df2_trimmed):
        raise ValueError("Trimmed dataframes do not have the same number of records after alignment.")

    print(f'Both dataframes trimmed to {len(df1_trimmed)} synchronized records.\n')

    # Columns to average (excluding 'Time')
    shared_columns = [col for col in df1_trimmed.columns if col in df2_trimmed.columns and col != 'Time']
    
    # Average data
    df_avg = pd.DataFrame()
    df_avg['Time'] = df1_trimmed['Time']
    for col in shared_columns:
        df_avg[col] = (df1_trimmed[col] + df2_trimmed[col]) / 2

    return df_avg


<br>

## 4. Shifted temperatures and humidex
Two new columns are added for the temperature and two new columns for humidex. One column corresponds to the instantaneous temperature/humidex difference between the sensor trajectory and the fixed reference sensor. The second column is the shifted temperature/humidex adjusted by adding the average fixed sensor temperature/humidex.

In [ ]:
def shifted_temperatures_and_humidex(df1, df_fixed):
    '''
    Recalculates temperature and humidex metrics relative to a fixed reference sensor.

    New columns added to `df1`:
        - 'T-T_fixed':          Instantaneous temperature difference.
        - 'T-T_fixed+<T>':      Temperature difference adjusted by average fixed sensor temperature.
        - 'HDX-HDX_fixed':      Instantaneous humidex difference.
        - 'HDX-HDX_fixed+<HDX>': Humidex difference adjusted by average fixed sensor humidex.
    
    Both input dataframes must have a 'Time' column and overlapping time ranges.

    Parameters:
        df1 (pd.DataFrame): Mobile sensor dataset.
        df_fixed (pd.DataFrame): Fixed reference sensor dataset.

    Returns:
        pd.DataFrame: Modified `df1` with new computed columns.
    '''
    
    # Ensure time columns are datetime
    df1['Time'] = pd.to_datetime(df1['Time'])
    df_fixed['Time'] = pd.to_datetime(df_fixed['Time'])

    # Get common time range
    start_time = max(df1['Time'].min(), df_fixed['Time'].min())
    end_time = min(df1['Time'].max(), df_fixed['Time'].max())

    print(f'Dataset 1: {len(df1)} records, from {df1["Time"].min()} to {df1["Time"].max()}')
    print(f'Dataset 2: {len(df_fixed)} records, from {df_fixed["Time"].min()} to {df_fixed["Time"].max()}')
    print(f'Common time window: {start_time} to {end_time}')

    # Trim both datasets to the common time range
    df1 = df1[(df1['Time'] >= start_time) & (df1['Time'] <= end_time)].reset_index(drop=True)
    df_fixed = df_fixed[(df_fixed['Time'] >= start_time) & (df_fixed['Time'] <= end_time)].reset_index(drop=True)

    if len(df1) != len(df_fixed):
        raise ValueError("Trimmed dataframes do not match in length. Cannot compute element-wise differences.")

    print(f'Both dataframes aligned with {len(df1)} matching records.\n')

    # Calculate differences and adjusted metrics
    df1['T-T_fixed'] = df1['Temp[°C]'] - df_fixed['Temp[°C]']
    df1['T-T_fixed+<T>'] = df1['T-T_fixed'] + df_fixed['Temp[°C]'].mean()

    df1['HDX-HDX_fixed'] = df1['HDX[°C]'] - df_fixed['HDX[°C]']
    df1['HDX-HDX_fixed+<HDX>'] = df1['HDX-HDX_fixed'] + df_fixed['HDX[°C]'].mean()

    return df1


<br>

## Example with a dataset

    - Date of the experiment: 10/07/2024
    - Using two sensors (sensor 1 and sensor 2) mounted in cart number 1. 
    - Using one fixed reference cart with 2 sensors (sensor 15 and sensor 17)

###  0. Read the data-sets

In [ ]:
# Cart 1 --> Sensor 1
df_cart1_sensor1 = pd.read_csv('Data_Repository_Heat_Chronicles\\original_trajectories\\SP_10july2024_cart_1_sensor_1.csv')  # Read csv file
df_cart1_sensor1.drop_duplicates(subset='Time', keep='first', inplace=True, ignore_index=True) # Remove possible duplicates
df_cart1_sensor1['Time'] = pd.to_datetime(df_cart1_sensor1['Time'], format='%Y-%m-%dT%H:%M:%S%z') # correct format
df_cart1_sensor1['Time'] = df_cart1_sensor1['Time'].dt.tz_localize(None) 

# Cart 1 --> Sensor 2
df_cart1_sensor2 = pd.read_csv('Data_Repository_Heat_Chronicles\\original_trajectories\\SP_10july2024_cart_1_sensor_2.csv')
df_cart1_sensor2.drop_duplicates(subset='Time', keep='first', inplace=True, ignore_index=True)
df_cart1_sensor2['Time'] = pd.to_datetime(df_cart1_sensor2['Time'], format='%Y-%m-%dT%H:%M:%S%z') 
df_cart1_sensor2['Time'] = df_cart1_sensor2['Time'].dt.tz_localize(None)


# Fixed cart --> sensor 15
df_fixed_cart_sensor15 = pd.read_csv('Data_Repository_Heat_Chronicles\\original_trajectories\\SP_10july2024_fixed_cart_sensor_15.csv')
df_fixed_cart_sensor15.drop_duplicates(subset='Time', keep='first', inplace=True, ignore_index=True)
df_fixed_cart_sensor15['Time'] = pd.to_datetime(df_fixed_cart_sensor15['Time'], format='%Y-%m-%dT%H:%M:%S%z') 
df_fixed_cart_sensor15['Time'] = df_fixed_cart_sensor15['Time'].dt.tz_localize(None)


# Fixed cart --> sensor 17
df_fixed_cart_sensor17 = pd.read_csv('Data_Repository_Heat_Chronicles\\original_trajectories\\SP_10july2024_fixed_cart_sensor_17.csv')
df_fixed_cart_sensor17.drop_duplicates(subset='Time', keep='first', inplace=True, ignore_index=True)
df_fixed_cart_sensor17['Time'] = pd.to_datetime(df_fixed_cart_sensor17['Time'], format='%Y-%m-%dT%H:%M:%S%z') 
df_fixed_cart_sensor17['Time'] = df_fixed_cart_sensor17['Time'].dt.tz_localize(None)

### 1. Remove columns

In [ ]:
df_cart1_sensor1_remove = remove_columns(df_cart1_sensor1)
df_cart1_sensor1_remove = remove_columns(df_cart1_sensor2)

df_fixed_cart_sensor15_remove = remove_columns(df_fixed_cart_sensor15)
df_fixed_cart_sensor17_remove = remove_columns(df_fixed_cart_sensor17)

### 2. Linear interpolation

In [ ]:
df_cart1_sensor1_interpolated = linear_interpolation(df_cart1_sensor1_remove)
df_cart1_sensor2_interpolated = linear_interpolation(df_cart1_sensor1_remove)

df_fixed_cart_sensor15_interpolated = linear_interpolation(df_fixed_cart_sensor15_remove)
df_fixed_cart_sensor17_interpolated = linear_interpolation(df_fixed_cart_sensor17_remove)

###  3. Average two sensors of the same trolley

In [ ]:
df_cart1_interpolated_averaged = average_2_sensors(df_cart1_sensor1_interpolated,df_cart1_sensor2_interpolated)

df_fixed_cart_interpolated_averaged = average_2_sensors(df_fixed_cart_sensor15_interpolated,df_fixed_cart_sensor17_interpolated)


### 4. Shifted temperatures and humidex

In [ ]:
df_cart1_interpolated_averaged_shifted = shifted_temperatures_and_humidex(df_cart1_interpolated_averaged,
                                                                          df_fixed_cart_interpolated_averaged)


### Save new data-frames

In [ ]:
#df_cart1_interpolated_averaged_shifted.to_csv('Data_Repository_Heat_Chronicles\\processed_trajectories\\SP_10july2024_cart_1.csv',index=False)
#df_fixed_cart_interpolated_averaged.to_csv('Data_Repository_Heat_Chronicles\\processed_trajectories\\SP_10july2024_fixed_cart.csv',index=False)